In [ ]:
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

### Test Local LangGraph Agent

In [ ]:
from langgraph_sdk import get_client

url = "https://leetmock001-76a7d6889223553c93a96358909dd6e3.default.us.langgraph.app"
client = get_client(url=url)

# Using the graph deployed with the name "agent"
assistant_id = "template"

# create thread
thread = await client.threads.create()

print(thread)

In [4]:
openai_assistant = await client.assistants.create(graph_id=assistant_id)

In [ ]:
openai_assistant

In [ ]:

async for chunk in client.runs.stream(
    thread["thread_id"],
    assistant_id,
    input={"messages": ["HI"], "event": "user_message"},
    stream_mode=["updates"],
    # multitask_strategy="interrupt",
):
    # tags = chunk.data.get("tags", [])
    # if "chatbot" not in tags:
    #     continue

    # event = chunk.data.get("event", None)
    # if event != "on_chat_model_stream":
    #     continue

    # content = chunk.data.get("data", {}).get("chunk", {}).get("content", "")

    print("-" * 100)
    print(chunk)
    print("-" * 100)

In [1]:
from agent_graph.template.graph import create_graph

graph = create_graph()

In [ ]:
async for chunk in graph.astream(
    input={
        "messages": ["HI"],
        "event": "user_message"
    },
    stream_mode=["updates"],
    config={ "configurable": {"thread_id": "1" } },
    subgraphs=True,
):
    print(chunk)

In [ ]:
from langchain_core.load.dump import dumps

state = graph.get_state(config={"configurable": {"thread_id": "1"}}, subgraphs=True)

print(dumps(state, pretty=True))

In [ ]:
from pydantic.v1 import BaseModel

class Test(BaseModel):
    a: int
    b: str

from langchain_core.prompts import PromptTemplate

PROMPT = """
{% for test in tests %}
{{ test.a }} {{ test.b }}
{% endfor %}
"""

prompt = PromptTemplate.from_template(PROMPT, template_format="jinja2")

print(
    prompt.invoke(
        {"tests": set([Test(a=1, b="2"), Test(a=3, b="4"), Test(a=5, b="6")])}
    ).to_string()
)

In [5]:
import asyncio
from debouncer import debounce

@debounce(wait=1.1)
def demo_function() -> None:
    print("called")

asyncio.create_task(demo_function())
await asyncio.sleep(1)
print("sleep 1")
asyncio.create_task(demo_function())
await asyncio.sleep(1)
print("sleep 2")
asyncio.create_task(demo_function())
await asyncio.sleep(1)
print("sleep 3")

sleep 1
sleep 2
sleep 3


called
